In [ ]:
import csv
import os
import requests

# Define the API endpoint and parameters
api_url = "https://api.openchargemap.io/v3/poi/"
params = {
    "output": "json",
    "countrycode": "AU",
    "maxresults": 1500,  # Increase max results to ensure all stations are retrieved
    "key": "36e92dd8-2f87-4ba2-909c-58f81f7ed881"  # Get your API key from Open Charge Map
}

# Make the API request
response = requests.get(api_url, params=params)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Specify CSV file path on the desktop
    desktop_path = os.path.join(os.path.expanduser("~/Desktop"), "capstoneB")
    os.makedirs(desktop_path, exist_ok=True)  # Create the folder if it doesn't exist
    csv_file_path = os.path.join(desktop_path, "EV_STATIONS.csv")

    try:
        # Open CSV file in write mode
        with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csv_file:
            # Create a CSV writer object
            csv_writer = csv.writer(csv_file)

            # Write header row
            csv_writer.writerow(["Title", "Latitude", "Longitude", "StateOrProvince", "Parking Status", "Usage Cost"])

            # Process the data and write rows to CSV
            for station in data:
                title = station.get("AddressInfo", {}).get("Title", "N/A")
                latitude = station.get("AddressInfo", {}).get("Latitude", "N/A")
                longitude = station.get("AddressInfo", {}).get("Longitude", "N/A")
                StateOrProvince = station.get("AddressInfo", {}).get("StateOrProvince", "N/A")
                # Check if "UsageType" exists and is not None
                if "UsageType" in station and station["UsageType"] is not None:
                    parking_status = station["UsageType"]["Title"]
                else:
                    parking_status = "Information not available"

                # Check if "UsageCost" exists and is not None
                if "UsageCost" in station and station["UsageCost"] is not None:
                    usage_cost = station["UsageCost"]
                else:
                    usage_cost = "Cost information not available"

                # Write row to CSV
                csv_writer.writerow([title, latitude, longitude,StateOrProvince, parking_status, usage_cost])

        print(f"CSV file saved successfully: {csv_file_path}")

    except Exception as e:
        print("Error occurred while creating the CSV file:", e)

else:
    print("Failed to fetch data. Status code:", response.status_code)